In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

import preprocessor as p
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv('clean_train.csv')
df.head()

,id,label,tweet
0,1,0,father dysfunctional selfish drag kid dysfunct...
1,2,0,thanks lyft credit cant use cause dont offer w...
2,3,0,bihday majesty
3,4,0,model love u take u time ur
4,5,0,factsguide society motivation


In [3]:
df = df.dropna()

In [4]:
data = df.sample(frac=0.5)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.30, random_state=42)

In [37]:
print(X_train.shape)
print(X_test.shape)

(11176,)
(4790,)


In [39]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=max_len)

In [41]:
print(X_train.shape)

(11176, 100)


In [42]:
sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(sequences, maxlen=max_len)
print(X_test.shape)

(4790, 100)


<p style='font-size: 30px;'><br><br><b>Embedding + Uni-Directional LSTM</b><p>

In [48]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

model1 = Sequential()
model1.add(layers.Embedding(max_words, 100)) #The embedding layer
model1.add(layers.LSTM(30,dropout=0.5)) #Our LSTM layer
model1.add(layers.Dense(64,activation='relu'))
model1.add(layers.Dense(32,activation='relu'))
model1.add(layers.Dense(16,activation='relu'))
model1.add(layers.Dense(1,activation='sigmoid'))

model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         500000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                1984      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_8 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 520,329
Trainable params: 520,329
Non-trainable params: 0
________________________________________________

In [51]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=2,validation_data=(X_test, y_test))

Epoch 1/2
350/350 [==============================] - 11s 28ms/step - loss: 0.0892 - accuracy: 0.9714 - val_loss: 0.1247 - val_accuracy: 0.9633
Epoch 2/2
350/350 [==============================] - 10s 27ms/step - loss: 0.0841 - accuracy: 0.9726 - val_loss: 0.1342 - val_accuracy: 0.9618


In [52]:
predictions = model1.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[4441   39]
 [ 144  166]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4480
           1       0.81      0.54      0.64       310

    accuracy                           0.96      4790
   macro avg       0.89      0.76      0.81      4790
weighted avg       0.96      0.96      0.96      4790

0.9617954070981211


<p style='font-size: 30px;'><br><br><b>Embedding + Bi-Directional LSTM</b><p>

In [53]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 100)) #The embedding layer
model2.add(layers.Bidirectional(layers.LSTM(30,dropout=0.6))) #Our LSTM layer
model2.add(layers.Dense(64,activation='relu'))
model2.add(layers.Dense(32,activation='relu'))
model2.add(layers.Dense(16,activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))

model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         500000    
_________________________________________________________________
bidirectional (Bidirectional (None, 60)                31440     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                3904      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 537,969
Trainable params: 537,969
Non-trainable params: 0
________________________________________________

In [54]:
model2.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=8,validation_data=(X_test, y_test))

Epoch 1/8
350/350 [==============================] - 16s 38ms/step - loss: 0.2647 - accuracy: 0.9285 - val_loss: 0.1686 - val_accuracy: 0.9455
Epoch 2/8
350/350 [==============================] - 13s 38ms/step - loss: 0.1676 - accuracy: 0.9423 - val_loss: 0.1534 - val_accuracy: 0.9509
Epoch 3/8
350/350 [==============================] - 14s 41ms/step - loss: 0.1484 - accuracy: 0.9508 - val_loss: 0.1413 - val_accuracy: 0.9553
Epoch 4/8
350/350 [==============================] - 14s 41ms/step - loss: 0.1248 - accuracy: 0.9587 - val_loss: 0.1371 - val_accuracy: 0.9589
Epoch 5/8
350/350 [==============================] - 14s 40ms/step - loss: 0.1201 - accuracy: 0.9599 - val_loss: 0.1291 - val_accuracy: 0.9610
Epoch 6/8
350/350 [==============================] - 14s 40ms/step - loss: 0.1119 - accuracy: 0.9640 - val_loss: 0.1275 - val_accuracy: 0.9616
Epoch 7/8
350/350 [==============================] - 14s 41ms/step - loss: 0.1142 - accuracy: 0.9644 - val_loss: 0.1259 - val_accuracy: 0.9626

In [56]:
predictions = model2.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[4438   42]
 [ 141  169]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      4480
           1       0.80      0.55      0.65       310

    accuracy                           0.96      4790
   macro avg       0.89      0.77      0.81      4790
weighted avg       0.96      0.96      0.96      4790

0.9617954070981211


<p style='font-size: 30px;'><br><br><b>Embedding + 1-D CNN</b><p>

In [59]:
model3 = Sequential()
model3.add(layers.Embedding(max_words, 100, input_length=max_len))
model3.add(layers.Conv1D(30, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.MaxPooling1D(5))
model3.add(layers.Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model3.add(layers.GlobalMaxPooling1D())
model3.add(layers.Dense(64,activation='relu'))
model3.add(layers.Dense(32,activation='relu'))
model3.add(layers.Dense(16,activation='relu'))
model3.add(layers.Dense(1,activation='sigmoid'))

model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 100)          500000    
_________________________________________________________________
conv1d (Conv1D)              (None, 95, 30)            18030     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 30)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 14, 20)            3620      
_________________________________________________________________
global_max_pooling1d (Global (None, 20)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                1344      
_________________________________________________________________
dense_15 (Dense)             (None, 32)               

In [63]:
model3.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
model3.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test), class_weight ={0:1.0, 1:10.0})

Epoch 1/10
350/350 [==============================] - 5s 13ms/step - loss: 1.2276 - acc: 0.8512 - val_loss: 0.5041 - val_acc: 0.7547
Epoch 2/10
350/350 [==============================] - 4s 12ms/step - loss: 0.8694 - acc: 0.7752 - val_loss: 0.4280 - val_acc: 0.8299
Epoch 3/10
350/350 [==============================] - 4s 12ms/step - loss: 0.7980 - acc: 0.8118 - val_loss: 0.3315 - val_acc: 0.9096
Epoch 4/10
350/350 [==============================] - 4s 12ms/step - loss: 0.6662 - acc: 0.8831 - val_loss: 0.3051 - val_acc: 0.9365
Epoch 5/10
350/350 [==============================] - 4s 12ms/step - loss: 0.5998 - acc: 0.9091 - val_loss: 0.3157 - val_acc: 0.9290
Epoch 6/10
350/350 [==============================] - 4s 12ms/step - loss: 0.5878 - acc: 0.9101 - val_loss: 0.2603 - val_acc: 0.9514
Epoch 7/10
350/350 [==============================] - 4s 12ms/step - loss: 0.5708 - acc: 0.9149 - val_loss: 0.3158 - val_acc: 0.9334
Epoch 8/10
350/350 [==============================] - 4s 12ms/step - 

In [64]:
predictions = model3.predict_classes(X_test)

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[4213  267]
 [  94  216]]
              precision    recall  f1-score   support

           0       0.98      0.94      0.96      4480
           1       0.45      0.70      0.54       310

    accuracy                           0.92      4790
   macro avg       0.71      0.82      0.75      4790
weighted avg       0.94      0.92      0.93      4790

0.9246346555323591
